<a href="https://colab.research.google.com/github/guten-morgen3776/NN-numpy/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np

def sigmoid_forward(z):
  sig = 1 / (1 + np.exp(-z))
  return sig

def relu_forward(z):
  relu = np.maximum(0, z)
  return relu



In [32]:
def initialize_parameters(n_x, n_h, n_y):
  """
  2層MLPのパラメータを初期化する
  n_x 入力層のサイズ
  n_h 隠れ層のサイズ
  n_y 出力層のサイズ
  """
  np.random.seed(2)
  w1 = np.random.randn(n_h, n_x) * 0.01
  b1 = np.zeros((n_h, 1))
  w2 = np.random.randn(n_y, n_h) * 0.01
  b2 = np.zeros((n_y, 1))

  parameters = {
      'w1': w1,
      'b1': b1,
      'w2': w2,
      'b2': b2
  }
  return parameters

params = initialize_parameters(2, 2, 1)
print(params)


{'w1': array([[-0.00416758, -0.00056267],
       [-0.02136196,  0.01640271]]), 'b1': array([[0.],
       [0.]]), 'w2': array([[-0.01793436, -0.00841747]]), 'b2': array([[0.]])}


In [33]:
def forward_propagetaion(X, parameters):

  w1 = parameters['w1']
  b1 = parameters['b1']
  w2 = parameters['w2']
  b2 = parameters['b2']

  Z1 = np.dot(w1, X) + b1
  A1 = relu_forward(Z1)

  Z2 = np.dot(w2, A1) + b2
  A2 = sigmoid_forward(Z2)

  cache = {
      'Z1': Z1,
      'A1': A1,
      'Z2': Z2,
      'A2': A2,
      'w1': w1,
      'w2': w2
  }
  return A2, cache


In [34]:
def backward_propagation(parameters, cache, X, Y):
  m = X.shape[1]

  w1 = cache['w1']
  w2 = cache['w2']
  A1 = cache['A1']
  A2 = cache['A2']
  Z1 = cache['Z1']

  #出力層
  dz2 = A2 - Y
  dw2 = np.dot(dz2, A1.T) / m #.Tは転置
  db2 = np.sum(dz2, axis=1, keepdims=True) / m #行列の和だから方向を指定
  #隠れ層
  dA1 = np.dot(w2.T, dz2)
  dz1 = np.array(dA1, copy=True)
  dz1[Z1 <= 0] = 0
  dw1 = np.dot(dz1, X.T) / m
  db1 = np.sum(dz1, axis=1, keepdims=True) / m

  grads = {
      'dw1': dw1,
      'db1': db1,
      'dw2': dw2,
      'db2': db2
  }
  return grads

In [35]:
def update_parameters(parameters, grads, learning_rate=0.01):
  w1 = parameters['w1']
  w2 = parameters['w2']
  b1 = parameters['b1']
  b2 = parameters['b2']
  dw1 = grads['dw1']
  dw2 = grads['dw2']
  db1 = grads['db1']
  db2 = grads['db2']

  lr = learning_rate

  w1 = w1 - lr * dw1
  w2 = w2 - lr * dw2
  b1  = b1 - lr * db1
  b2 = b2 - lr * db2

  new_params = {
      'w1': w1,
      'w2': w2,
      'b1': b1,
      'b2': b2
  }
  return new_params


In [36]:
import matplotlib.pyplot as plt

def compute_cost(A2, Y):
  m = Y.shape[1]
  logprobs = np.multiply(np.log(A2 + 1e-15), Y) + np.multiply((1 - Y), np.log(1 - A2 + 1e-15))
  #０で割ることを防ぐために微少値を入れる
  #np.multiply 配列を受け取って要素ごとの積を計算
  cost = - np.sum(logprobs) / m
  return cost

def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False, learning_rate=1.2):
  np.random.seed(3)
  n_x = X.shape[0]
  n_y = Y.shape[0]

  parameters = initialize_parameters(n_x, n_h, n_y)

  for i in range(num_iterations):
    A2, cache = forward_propagetaion(X, parameters)
    cost = compute_cost(A2, Y)
    grads = backward_propagation(parameters, cache, X, Y)
    parameters = update_parameters(parameters, grads, learning_rate)
    if print_cost and i % 1000 == 0:
      print(f'cost after iteration {i}: {cost}')
  return parameters




In [38]:
X = np.array([[0, 0, 1, 1],
             [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])

parameters = nn_model(X, Y, n_h=4, num_iterations=10000, print_cost=True)
A2, _ = forward_propagetaion(X, parameters)
print(A2)


cost after iteration 0: 0.6931514105429224
cost after iteration 1000: 0.001324869376508153
cost after iteration 2000: 0.0006151993058231423
cost after iteration 3000: 0.00039833342009968937
cost after iteration 4000: 0.0002933164711151205
cost after iteration 5000: 0.00023174493433106486
cost after iteration 6000: 0.00019134007016449476
cost after iteration 7000: 0.00016285041487219498
cost after iteration 8000: 0.00014166047955030186
cost after iteration 9000: 0.000125308101033361
[[1.97906879e-04 9.99973330e-01 9.99973373e-01 1.97906879e-04]]
